In [0]:
# ===================================================================
# CELL 1: Install Packages (Development)
# ===================================================================

%pip install \
    mlflow[databricks]>=2.16.0 \
    databricks-agents \
    databricks-openai \
    openai \
    pydantic \
    unitycatalog-ai \
    uv \
    --upgrade --quiet

dbutils.library.restartPython()

## Clean up unsuccessfully deployed versions

In [0]:
# ===================================================================
# CELL 2: Clean Up Old Versions (Optional)
# ===================================================================

from mlflow.tracking import MlflowClient

client = MlflowClient()
UC_MODEL_NAME = "dev_kiddo.silver.CareGapsModel"

# Delete failed conda versions
failed_versions = [8, 9, 10, 11, 12, 13, 14]

for version in failed_versions:
    try:
        client.delete_model_version(UC_MODEL_NAME, str(version))
        print(f"✓ Deleted version {version}")
    except Exception as e:
        print(f"  Version {version}: already deleted or doesn't exist")

print("\n✓ Cleanup complete")

# LOG Model with pre-deployent check

In [0]:
# ===================================================================
# VALIDATED DEPLOYMENT WITH PRE-DEPLOYMENT CHECK
# ===================================================================

import mlflow
from databricks.sdk import WorkspaceClient

mlflow.set_registry_uri("databricks-uc")

UC_MODEL_NAME = "dev_kiddo.silver.CareGapsModel"
ENDPOINT_NAME = "agents_dev_kiddo-silver-CareGapsModel"

# Find agent.py
import os
if os.path.exists("agent.py"):
    agent_path = "agent.py"
else:
    agent_path = "/Workspace/Users/adminjkhan@akronchildrens.org/agent.py"

print(f"{'='*60}")
print("STEP 1: LOG MODEL")
print('='*60)

with mlflow.start_run(run_name="validated_deployment"):
    
    model_info = mlflow.pyfunc.log_model(
        artifact_path="agent",
        python_model=agent_path,
        registered_model_name=UC_MODEL_NAME,
        pip_requirements=[
            "mlflow[databricks]>=2.16.0",
            "databricks-openai>=0.2.0",
            "openai>=1.0.0",
            "pydantic>=2.0.0",
            "unitycatalog-ai>=0.1.0",
        ],
        input_example={
            "input": [{"role": "user", "content": "How many care gaps?"}]
        }
    )
    
    run_id = model_info.run_id
    version = model_info.registered_model_version
    
    print(f"✓ Model logged")
    print(f"  Version: {version}")
    print(f"  Run ID: {run_id}")

## Run validation test to see if model is returning responses

In [0]:
# ===================================================================
# FIXED Validation Script
# ===================================================================

import mlflow
from mlflow.tracking import MlflowClient

mlflow.set_registry_uri("databricks-uc")
client = MlflowClient()

# Get latest model
model_versions = client.search_model_versions("name='dev_kiddo.silver.CareGapsModel'")
latest = max(model_versions, key=lambda x: int(x.version))
run_id = latest.run_id

print(f"Testing model from run: {run_id}\n")

test_queries = [
    "How many care gaps are there?",
    "Show me critical gaps",
    "Find patient 2886348",
]

all_passed = True

for i, query in enumerate(test_queries, 1):
    print(f"Test {i}/{len(test_queries)}: {query}")
    try:
        response = mlflow.models.predict(
            model_uri=f"runs:/{run_id}/agent",
            input_data={"input": [{"role": "user", "content": query}]},
            env_manager="uv",
        )
        print(f"  ✓ Passed")
        print(f"  Preview: {full_content[:150]}...")
            
    except Exception as e:
        print(f"  ✗ FAILED: {e}")
        import traceback
        traceback.print_exc()
        all_passed = False
    
    print()

print("="*60)
if all_passed:
    print("✓✓✓ ALL TESTS PASSED ✓✓✓")
else:
    print("⚠️  VALIDATION HAD ISSUES")
    print("But agent may still be working - check output above")
print("="*60)

## Deploy the agent as serving endpoint

In [0]:
# ============================================================
# STEP 3: DEPLOY TO SERVING ENDPOINT
# ============================================================

from databricks.sdk import WorkspaceClient
from databricks.sdk.service.serving import ServedEntityInput, EndpointCoreConfigInput
from mlflow.tracking import MlflowClient

mlflow.set_registry_uri("databricks-uc")

# Configuration
UC_MODEL_NAME = "dev_kiddo.silver.CareGapsModel"
ENDPOINT_NAME = "agents_dev_kiddo-silver-CareGapsModel"

print(f"{'='*60}")
print("DEPLOYING AGENT TO SERVING ENDPOINT")
print('='*60)

# Get latest registered version
client = MlflowClient()
model_versions = client.search_model_versions(f"name='{UC_MODEL_NAME}'")
latest_version = max(model_versions, key=lambda x: int(x.version))

print(f"\nModel: {UC_MODEL_NAME}")
print(f"Version: {latest_version.version}")
print(f"Endpoint: {ENDPOINT_NAME}\n")

# Create WorkspaceClient
w = WorkspaceClient()

# Create served entity using SDK class
served_entity = ServedEntityInput(
    entity_name=UC_MODEL_NAME,
    entity_version=str(latest_version.version),
    workload_size="Small",
    scale_to_zero_enabled=False,
)

print("Deploying...")

try:
    # Try to update existing endpoint
    w.serving_endpoints.update_config(
        name=ENDPOINT_NAME,
        served_entities=[served_entity],
    )
    print(f"✓ Updated existing endpoint")
    
except Exception as e:
    error_msg = str(e)
    
    if "RESOURCE_DOES_NOT_EXIST" in error_msg or "does not exist" in error_msg.lower():
        # Create new endpoint if it doesn't exist
        print("Endpoint doesn't exist, creating...")
        
        w.serving_endpoints.create(
            name=ENDPOINT_NAME,
            config=EndpointCoreConfigInput(
                name=ENDPOINT_NAME,
                served_entities=[served_entity]
            )
        )
        print(f"✓ Created new endpoint")
    else:
        print(f"✗ Deployment failed: {error_msg}")
        raise

print(f"\n{'='*60}")
print("✓✓✓ DEPLOYMENT INITIATED ✓✓✓")
print('='*60)
print(f"\nEndpoint: {ENDPOINT_NAME}")
print(f"Version: {latest_version.version}")
print(f"\nContainer build will take 3-5 minutes...")
print(f"Monitor at: Serving > Endpoints > {ENDPOINT_NAME}")